In [1]:
import cpuinfo
cpuinfo.get_cpu_info()['brand']

'Intel(R) Core(TM) i3-6100U CPU @ 2.30GHz'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17528890222903738202, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11141117092253367110
 physical_device_desc: "device: XLA_CPU device"]

In [3]:
import tensorflow as tf
tf.test.is_gpu_available()

False

In [4]:
tf.test.gpu_device_name()

''

In [5]:
import imageio
from tqdm import tqdm
import multiprocessing as mp
import pandas as pd
import seaborn as sns

In [6]:
%run ../utils/unet.py
%run ../utils/helpers.py
%run ../utils/generate_flat_bg.py
%run ../utils/generate_bg.py
%run ../utils/generate_train_set.py

Using TensorFlow backend.


In [7]:
h_size = 256
w_size = 256
start_neurons = 16
mask_pool_size = 100
generate_batch_size = 20
train_num = 100
bg_admix_iters = 2

cores_count = mp.cpu_count()
# cpu_pool = mp.Pool(processes=cores_count)

In [8]:
train_x = np.zeros((train_num, h_size, w_size, 3), dtype='float32')
train_y = np.zeros((train_num, h_size, w_size, 1), dtype='float32')

model = get_compiled_model(h_size, w_size, start_neurons)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
sample_img = imageio.imread('../data/train/original_sample_blur_1000.jpg', pilmode="RGB")
mask_img = imageio.imread('../data/train/original_sample_mask_inv_1000.jpg', pilmode="RGB", as_gray=True)

original_smpl = np.array(sample_img, dtype=np.uint8)
original_mask = np.array(mask_img, dtype=np.uint8)

In [10]:
simplest_flat_bg = imageio.imread('../data/train/simplest_flat_bg.jpg', pilmode="RGB")
simple_bg = imageio.imread('../data/train/simplest_bg.jpg', pilmode="RGB")

In [11]:
mask_pool = [
    get_mask(h_size, w_size) 
    for _ in tqdm(range(mask_pool_size))
]

100%|██████████| 100/100 [00:07<00:00, 15.29it/s]


In [12]:
def get_next_batch(
    original_mask,
    original_smpl,
    simplest_flat_bg,
    simple_bg,
    mask_pool,
    batch_size,
):
    return [
        next_pair(
            original_mask,
            original_smpl,
            simplest_flat_bg,
            simple_bg,
            mask_pool,
        )
        for _ in range(batch_size)
    ]

In [23]:
%%time
generate_batches_count = int(train_num / generate_batch_size)
with mp.Pool(processes=cores_count) as pool:
    results = [
        pool.apply_async(
            get_next_batch, 
            args=(
                original_mask,
                original_smpl,
                simplest_flat_bg,
                simple_bg,
                mask_pool,
                generate_batch_size,
            )
        ) 
        for _ in range(generate_batches_count)
    ]
    k = 0
    batch_n = 0
    for list_pairs in [res.get() for res in results]:
        print('Batch %s complete' % batch_n)
        batch_n += 1
        for img, msk in list_pairs:
            train_x[k] = img
            train_y[k] = msk
            k += 1

Batch 0 complete
Batch 1 complete
Batch 2 complete
Batch 3 complete
Batch 4 complete
CPU times: user 242 ms, sys: 602 ms, total: 844 ms
Wall time: 17.3 s


In [24]:
%%time
with mp.Pool(processes=cores_count) as cpu_pool:
    coros = [
        cpu_pool.apply_async(
            next_pair, 
            args=(
                original_mask,
                original_smpl,
                simplest_flat_bg,
                simple_bg,
                mask_pool,
            )
        ) 
        for _ in range(train_num)
    ]
    k = 0
    for img, msk in [p.get() for p in coros]:
        train_x[k] = img
        train_y[k] = msk
        k += 1

CPU times: user 2.33 s, sys: 3.89 s, total: 6.22 s
Wall time: 16.1 s


In [15]:
# acc = []
# val_acc = []
# for _ in range (50):
#     history = model.fit(train_x, train_y,
#                         batch_size=32,
#                         epochs=1,
#                         verbose=1,
#                         validation_split=0.1
#                        )    
#     acc_ = history.history['my_iou_metric'][0]
#     val_acc_ = history.history['val_my_iou_metric'][0]
#     acc.append(acc_)
#     val_acc.append(val_acc_)
#     if acc_ > 0.99:
#         break

In [16]:
# df = pd.DataFrame({'x': list(range(len(acc))), 'y': acc})
# sns.lmplot(x="x", y="y", data=df, order=2);

In [17]:
# df = pd.DataFrame({'x': list(range(len(val_acc))), 'y': val_acc})
# sns.lmplot(x="x", y="y", data=df, order=2);